## Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('cruise').getOrCreate()
data  = spark.read.csv('/FileStore/tables/cruise_ship_info.csv', inferSchema=True, header=True)
data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [3]:
# Dealing with the Cruise_line categorical variable

data.groupBy('Cruise_line').count().show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [4]:
# StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.

from pyspark.ml.feature import StringIndexer

stringIndexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_cat")
index_df = stringIndexer.fit(data).transform(data)
index_df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
only showing top 20 rows

In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

asssembler = VectorAssembler(inputCols = ['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'Cruise_cat'],
                            outputCol = 'features')
output = asssembler.transform(index_df)
output.select("features", "crew").show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [6]:
# train test split
train_data, test_data = output.randomSplit([0.7, 0.3])

In [7]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol = 'crew')
lrModel = lr.fit(train_data)

print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.014706255010697862,0.0153840571006578,-0.15517327386982,0.43095755833981786,0.8049787121028131,-0.014006014452068255,0.045353470381098916] Intercept: -0.49525564141612394

In [8]:
test_results = lrModel.evaluate(test_data)

print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 1.2856785851991865
MSE: 1.652969424439782
R2: 0.8025609464341773

In [9]:
# R2 of 0.86 is pretty good, let's check the data a little closer
from pyspark.sql.functions import corr

data.select(corr('crew','passengers')).show()
data.select(corr('crew','cabins')).show()

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+